In [1]:
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from tqdm.notebook import tqdm
import math
import scipy as sp
import random

In [2]:
data = pd.read_csv('cleandataLL.csv')

In [3]:
df = data[["sub", "lessonType", "cue_text", "answer", "reactionTime", "correct", "givenResponse", "repetition"]]

In [4]:
df = df.rename(columns={'sub': 'participant', 'lessonType':'condition', 'cue_text':'cue', 'answer':'target','givenResponse':'response', 'reactionTime':'rt'})

In [5]:
# Change column values
df['condition'] = np.where(df['condition'].str.strip() == 'Recall', 2, 1)
df['correct'] = np.where(df['correct'] == True, 1, 0)
df['rt'] = df['rt']*1000

In [6]:
df.head()

,participant,condition,cue,target,rt,correct,response,repetition
0,sub01,2,Zawadi,Reward,1577.0,1,dog,0
1,sub01,2,Bustani,Garden,4337.0,1,broom,0
2,sub01,2,Kitambaa,Fabric,1645.0,1,cloud,0
3,sub01,2,Dini,Religion,1156.0,1,religion,0
4,sub01,2,Sumu,Poison,1505.0,1,poison,0


In [7]:
def activation(traces, time, decay):
    """Computes the activation of a memory given its history of retrievals"""
    ftraces = [x for x in traces if x < time]
    decay = max(0, decay)  # Allows no positive decay rates in equation 
    decay - min(decay, 5)
    times = time - np.array(ftraces)
    odds = times ** -decay
    return np.log(np.sum(odds))

In [8]:
def boltzmann(options, values, temperature):
    """Returns a Boltzmann distribution of the probabilities of each option"""
    temperature = max(temperature, 0.01) 
    vals = np.array(values)/temperature
    #bvals = np.exp(vals)/np.sum(np.exp(vals))
    bvals = np.exp(vals - np.max(vals)) / np.exp(vals - np.max(vals)).sum()
    return dict(zip(options, bvals))

In [9]:
def responsetime(activation, ter, F=1, f=1):
    return ter + F * np.exp(-f * activation)

In [10]:
from urllib.request import proxy_bypass
def rtProb(rt, activation, s, ter):
  """Takes one parameter for noise, s, and outputs a probability distribution for response times"""
  noise = np.linspace(-2, 2)
  dist = sp.stats.logistic(0, ((math.pi**2)*s)/3)
  rts = [responsetime((activation - x), ter) for x in noise]
  prob = dist.pdf(noise)
  rtprob = {rts[i]:prob[i]for i in range(len(noise))}
  val = min(rtprob.keys(), key=lambda x: abs(x - (rt/1000)))
  return rtprob[val]

In [93]:
def LLelabRT(alldata, ppt, rep, decay, temp, ter, mas = 1.6):
    """For each trial, calculate the probability of that response, sum the log likelihoods, and update the values"""
    data = alldata[(alldata.participant == ppt) & (alldata.repetition == rep)]

    # Sample min_count rows for each condition
    sampled_1 = data[data['condition'] == 1].sample(n=25)
    sampled_2 = data[data['condition'] == 2].sample(n=25)

    # Combine the samples
    data = pd.concat([sampled_1, sampled_2])

    # create a list of error items
    errors = data[data.condition == 1].cue.tolist()
    # create a list of study items
    study = data[data.condition == 2].cue.tolist()
    
    pos = 1
    present = errors[:]
    for i in range(len(errors)):
      word = study[i]
      present.insert(pos, word)
      pos += 2

    # make sure we have "some" data to move on
    if len(present) >= 50:
      # Create dict with word pairs
      pairs = {}
      for cue, target in zip(data.cue,data.target):
        pairs[cue] = target
      # also create a dict with errors
      errorResp = dict()
      for cue,response in zip(alldata[(alldata.participant == ppt) & (alldata.condition == 1) & (alldata.repetition == 0)].cue,
                              alldata[(alldata.participant == ppt) & (alldata.condition == 1) & (alldata.repetition == 0)].response):
        errorResp[cue] = response

      # model learning phase, encode a single trace for each item: (idk what to set activation at)
      DM = dict()
      # for DM can we make a dictionary of dictionaries where big keys are cues, values are dictionary of target/
      # possible responses and their activation
      time = 0
      for cue in present:
        littleDM = {}
        # make a set of all reponses given to a certain cue to be "vocab for that cue"
        for response in set(alldata[alldata.cue == cue].response):
          littleDM[response] = [0.001]
        # add retrieval of error for error items
        if cue in errorResp.keys():
          error = errorResp[cue]
          time +=5
          littleDM[error] = [0.001, time]
        # overwrite smaller activ of correct target to show task learning
          time +=5
          littleDM[pairs[cue]] = [0.001, time]

        else:
          time += 10
          littleDM[pairs[cue]] = [0.001, time]

        DM[cue] = littleDM
      time += 300 # time for distractor phase


      # model testing phase
      LL = 0
      for condition, cue, response, rt, feedback in zip(data.condition,
                                    data.cue, 
                                    data.response, 
                                    data.rt, 
                                    data.correct):
        # Calculate log likelihood of response- possible options are 19 random integers
        # or correct associate
        options = DM[cue].keys()
        # create spreading activation additional error component given size of cue's dec mem
        cueMem = len(DM[cue])
        add = (mas - np.log((cueMem + 1)/2)) - (mas - np.log((cueMem + 1)/1))

        # if error condition, add spreading activation
        values = [(activation(DM[cue][opt], time, decay) + add) if condition == 1 else 
        activation(DM[cue][opt], time, decay) for opt in options]
        prob = boltzmann(options, values, temp)[response]
      
        # now calculate response times:
        if condition == 1:
          resp_activation = activation(DM[cue][response], time, decay) + add
        else: 
          resp_activation = activation(DM[cue][response], time, decay)

        
        prob_rt = rtProb(rt, resp_activation, temp, ter)
        print(prob_rt)
        # Sum up the LLs
        # LL += (np.log(max(prob, 10e-10)) + np.log(max(prob_rt, 10e-10)))
        LL += np.log(max(prob_rt, 10e-10))

        # add time taken to responde
        time += rt/1000
    else:
      LL = 0
    return LL
  

In [94]:
print(LLelabRT(df, ppt='sub01', rep=0, decay=0.5, temp=1, ter=1))
print(LLelabRT(df, ppt='sub01', rep=3, decay=0.5, temp=1, ter=1))

0.07446472084049492
0.07591782870035427
0.07471575050394451
0.07446472084049492
0.06938066766236886
0.0721241220126985
0.07549882664815326
0.07596457568570096
0.06938066766236886
0.06938066766236886
0.07127971677450015
0.07251833954903393
0.07446472084049492
0.07584778011050178
0.06988043632706319
0.07596457568570099
0.07389836349880202
0.0721241220126985
0.07563815067451198
0.06938066766236886
0.07289329429984732
0.06938066766236886
0.07446472084049492
0.07515201002448926
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.07251833954903393
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
-1

In [30]:
from urllib.request import proxy_bypass
def rtProb2(rt, resp_activation, error_activation, condition, s, ter):
  """Takes one parameter for noise, s, and outputs a probability distribution for response times"""
  noise = np.linspace(-2, 2)
  dist = sp.stats.logistic(0, ((math.pi**2)*s)/3)
  if condition == 1:
    rts = [(responsetime((resp_activation - x), ter) + responsetime((error_activation - x), ter)) for x in noise]
  else: 
    rts = [responsetime((resp_activation - x), ter) for x in noise]
  prob = dist.pdf(noise)
  rtprob = {rts[i]:prob[i]for i in range(len(noise))}
  val = min(rtprob.keys(), key=lambda x: abs(x - (rt/1000)))
  return rtprob[val]

In [95]:
def LLmedRT(alldata, ppt, rep, decay, temp, ter):
    """For each trial, calculate the probability of that response, sum the log likelihoods, and update the values"""
    data = alldata[(alldata.participant == ppt) & (alldata.repetition == rep)]

    # Sample min_count rows for each condition
    sampled_1 = data[data['condition'] == 1].sample(n=25)
    sampled_2 = data[data['condition'] == 2].sample(n=25)

    # Combine the samples
    data = pd.concat([sampled_1, sampled_2])

    # create a list of error items
    errors = data[data.condition == 1].cue.tolist()
    # create a list of study items
    study = data[data.condition == 2].cue.tolist()
    
    pos = 1
    present = errors[:]
    for i in range(len(errors)):
      word = study[i]
      present.insert(pos, word)
      pos += 2

    #check for some data
    if len(present) >= 50:
      # Create dict with word pairs
      pairs = {}
      for cue, target in zip(data.cue,data.target):
        pairs[cue] = target
      # also create a dict with errors
      errorResp = dict()
      for cue,response in zip(data[data.condition == 1].cue,data[data.condition == 1].response):
        errorResp[cue] = response

      # model learning phase, encode a single trace for each item: (idk what to set activation at)
      DM = dict()
      # for DM can we make a dictionary of dictionaries where big keys are cues, values are dictionary of target/
      # possible responses and their activation
      time = 0
      step = 10 #time for learning each item
      for cue in present:
        littleDM = {}
        # make a set of all reponses given to a certain cue to be "vocab for that cue"
        for response in set(alldata[alldata.cue == cue].response):
          littleDM[response] = [0.001]
        # add retrieval of error for error items
        if cue in errorResp.keys():
          error = errorResp[cue]
          time +=5
          littleDM[error] = [0.001, time]
        # overwrite smaller activ of correct target to show task learning
          time +=5
          littleDM[pairs[cue]] = [0.001, time]

        else:
          time += 10
          littleDM[pairs[cue]] = [0.001, time]

        DM[cue] = littleDM
      time += 300 # time for distractor phase

      # model testing phase
      LL = 0
      
      for condition, cue, response, rt, feedback in zip(data.condition,
                                    data.cue, 
                                    data.response, 
                                    data.rt, 
                                    data.correct):
          # Calculate log likelihood of response- possible options are 19 random integers
          # or correct associate
          options = DM[cue].keys()

          # calculate probability of retrieving given response
          values = [activation(DM[cue][opt], time, decay) for opt in options]
          prob1 = boltzmann(options, values, temp)[response]
          
          # probability of retrieving error memory
          if condition == 1:
            error = errorResp[cue]
            prob2 = boltzmann(options, values, temp)[error]
          else:
            prob2 = 0
          
          # add response times calculations
          # probability of given response time with
          respAct = activation(DM[cue][response], time, decay)
          if condition == 1:
            error = errorResp[cue]
            errorAct = activation(DM[cue][error], time, decay)
            prob_rt = rtProb2(rt, respAct, errorAct, condition, temp, ter)
          else:
            errorAct = 0
            prob_rt = rtProb2(rt, respAct, errorAct, condition, temp, ter)

          # Sum up the LLs
          # LL += (np.log(max((prob1 + prob2)/1.4, 10e-10)) + np.log(max(prob_rt, 10e-10)))
          LL += np.log(max(prob_rt, 10e-10))
          print(prob_rt)
          # add time taken to responde
          time += rt/1000   
    else:
      LL = 0     
    return LL

In [96]:
print(LLmedRT(df, ppt='sub01', rep=0, decay=0.5, temp=1, ter=1))
print(LLmedRT(df, ppt='sub01', rep=2, decay=0.5, temp=1, ter=1))

0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.07127971677450015
0.06988043632706319
0.06938066766236886
0.06938066766236886
0.07127971677450015
0.06938066766236886
0.0721241220126985
0.06938066766236886
0.07324855457003711
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.07251833954903393
0.07083048066308276
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
0.06938066766236886
-

In [90]:
def vLLelab(array, data, ppt, rep):
    """Vector function of procedural log-likelihood"""
    decay, temp, ter = array
    return -1 * LLelabRT(data, ppt, rep, decay, temp, ter)


def vLLmed(array, data, ppt, rep):
    """Vector function of procedural log-likelihood"""
    decay, temp, ter = array
    return -1 * LLmedRT(data, ppt, rep, decay, temp, ter)

In [103]:
DO_ALL = True
dataframe = []

if DO_ALL:
    for ppt in df.participant.unique():
        for rep in range(10):
            print(ppt, rep)
            data = df[(df.participant == ppt) & (df.repetition == rep)]
            edecay, etemp, eter = opt.minimize(vLLelab, x0 = [0.5, 1, 1], args=(data, ppt, rep), method = "Powell", bounds=[[0.01, 2], [0, 2], [0.1, 2]]).x
            llelab = LLelabRT(df, ppt, rep, edecay, etemp, eter)
                
            mdecay, mtemp, mter = opt.minimize(vLLmed, x0 = [0.5, 1, 1], args=(data, ppt, rep), method = "Powell", bounds=[[0.01, 2], [0, 2], [0.1, 2]]).x
            llmed = LLmedRT(df, ppt, rep, mdecay, mtemp, mter)
                
            best = "Mediator"
            if llelab > llmed:
                best = "Elaborative"

            print(best)    
            diff = llmed - llelab
                
            row = [ppt, rep, edecay, etemp, eter, llelab, mdecay, mtemp, mter, llmed, best, diff]
                
            dataframe += [row]

        result = pd.DataFrame(dataframe, columns=["Participant", "Presentation", "elab.decay", "elab.temp", "elab.ter", "elab.LL", "med.decay", "med.temp", "med.ter", "med.LL", "best.model", "diff.LL"])
        result.to_csv('LL_results.csv')

sub01 0
Mediator
sub01 1
Mediator
sub01 2
Mediator
sub01 3
Mediator
sub01 4
Mediator
sub01 5
Mediator
sub01 6
Mediator
sub01 7
Mediator
sub01 8
Mediator
sub01 9
Mediator
sub02 0
Mediator
sub02 1


c:\Users\Bridget Leonard\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\stats\_distn_infrastructure.py:1983: RuntimeWarning: divide by zero encountered in divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)


Mediator
sub02 2
Mediator
sub02 3
Mediator
sub02 4
